In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import os
import cv2
import sys
import math
from pathlib import Path
PIPELINE_ROOT = Path('./').resolve().parent.parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())
print(PIPELINE_ROOT)

from src.library.utilities.utilities_process import read_image, write_image

In [ ]:
animal = 'DK55'
section = 172
channel = 3
tif = str(section).zfill(3) + ".tif"
prep_path = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'
filepath = os.path.join(prep_path, f'C{channel}/full_aligned', tif)
full_img = read_image(filepath)
full_img.shape

In [ ]:
# min
#x          34189.538462
#y          10960.000000
# max
#x          44627.384615
#y          23173.200000
# for using tile = 5, we push the origin down 2 tile_heights

dfpath = os.path.join(prep_path, 'cell_labels', '057.csv')
df = pd.read_csv('172.csv')
#df = df.loc[df['section'] == section]
print(len(df.index))
df.head()

In [ ]:
startc = int(round(min(df['x']), -3))
startr = int(round(min(df['y']), -3))
endc = math.ceil(max(df['x']))
endr = math.ceil(max(df['y']))
print(startr, endr, startc, endc)

In [ ]:
tile_width = endc - startc
tile_height = endr - startr
img = full_img[startr:endr, startc:endc]
print(f'{startr}:{endr}, {startc}:{endc} height={tile_height} width={tile_width}')
print(img.shape)
#img.shape # (6800, 30000)

In [ ]:
vimg = img.copy()
def show_image(img, increaser=1, _title="", fsize=[15,15]):
    plt.figure(figsize=[15,15])
    plt.imshow(img * increaser, cmap="gray")
    plt.title(_title,fontsize=20)
    plt.tick_params(axis='x', labelsize=20)
    plt.tick_params(axis='y', labelsize=20)
    plt.show()
increaser = 10
show_image(vimg, increaser=increaser,_title=f'{animal} C{channel}, section: {section}')

In [ ]:
def collect_positives(uimg):
    img=np.copy(uimg)
    positives=[]
    radius = 80
    a=int(radius/2)
    color = (65000)
    min_row,min_col = startr, startc
    for index,r in df.iterrows():
        col = int(round(r['x'])-min_col)
        row = int(round(r['y'])-min_row)
        if (col<0) or (col>tile_width) or (row<0) or (row>tile_height):
            continue
        img=cv2.circle(img, (int(col), int(row)), radius, color, 20)
        I=uimg[row-a:row+a,col-a:col+a]
        if I.shape==(radius,radius):
            positives.append((col,row,I))

    return img,positives

In [ ]:
annotated_img,positives=collect_positives(img)

In [ ]:
#vimg=annotated_img[in_row_min:in_row_max, in_col_min: in_col_max]
show_image(annotated_img, increaser=increaser,_title=f' annotated {animal} C{channel}, section: {section}')

In [ ]:
plt.figure(figsize=[15,15])
for i in range(len(positives[:100])):
    plt.subplot(10,10,i+1)
    plt.imshow(positives[i][2], cmap="gray")
plt.show()

In [ ]:
def plot_dist(uimg):
    H=uimg.flatten()
    H=np.sort(H)

    H=H[0:-1:1000]
    print(H.shape)
    H1 = np.arange(0,1,1./H.shape[0])
    H1 = H1[:H.shape[0]]
    print(H1.shape)

    plt.plot(H, H1)
    plt.grid()
    plt.figure()
    plt.hist(H,bins=100);
plot_dist(img)

In [ ]:
for i in [10,20,30,40]:
    plt.hist(positives[i][2].flatten(),bins=100);

In [ ]:
for i in range(len(positives)):
    _shape=positives[i][2].shape
    if _shape !=(80,80):
        print(i,_shape)

In [ ]:
S=np.stack([I for (x,y,I)in positives])
S.shape

Mean=np.mean(S,axis=0)
Mean.shape

plt.imshow(Mean, cmap="gray")
plt.show()

_mean=np.mean(Mean.flatten())
_std=np.std(Mean.flatten())
print('before: mean=%5.2f std=%5.2f'%(_mean,_std))

Mean = (Mean-_mean)/_std

_mean=np.mean(Mean.flatten())
_std=np.std(Mean.flatten())
print('after: mean=%5.2f std=%5.2f'%(_mean,_std))

import pickle as pk
with open('mean.pkl','wb') as pkl_file:
    pk.dump({'Mean':Mean},pkl_file)

!ls -l *.pkl

In [ ]:
from scipy.signal import convolve2d

In [ ]:
#O=convolve2d(img,Mean,mode='valid') #takes forevr, might be better with GPU

In [ ]:
plt.figure(figsize=[15,15])
UPimg=uimg[in_row_min:in_row_max, in_col_min: in_col_max]
plt.imshow(5*UPimg,cmap='gray');

In [ ]:
plt.figure(figsize=[15,15])
small=cv2.resize(S,(0,0),fx=0.05,fy=0.05)
show_image(small);

In [ ]:
blur=cv2.GaussianBlur(small,ksize=(21,21),sigmaX=10)
relarge=cv2.resize(blur,(0,0),fx=20,fy=20) #,interpolation=cv2.INTER_AREA)
plt.figure(figsize=[15,15])
plt.imshow(relarge,cmap='gray');

In [ ]:
relarge=cv2.resize(blur,(0,0),fx=20,fy=20) #,interpolation=cv2.INTER_AREA)
plt.figure(figsize=[15,15])
plt.imshow(relarge,cmap='gray');

In [ ]:
blur.shape,uimg_8bit.shape,relarge.shape

In [ ]:
vimg=img[in_row_min:in_row_max, in_col_min: in_col_max]
show_image(5*vimg)

In [ ]:
diff=S-relarge
plt.figure(figsize=[15,15])
show_image(diff[in_row_min:in_row_max, in_col_min: in_col_max]);

In [ ]:
plt.figure(figsize=[15,15])
vdiff=diff[in_row_min:in_row_max, in_col_min: in_col_max]
show_image(vdiff>150);

In [ ]:
plot_dist(diff[in_row_min:in_row_max, in_col_min: in_col_max])

In [ ]:
Stats=cv2.connectedComponentsWithStats(np.int8(diff>130))
Stats[0]

In [ ]:
show_image(Stats[1][in_row_min:in_row_max, in_col_min: in_col_max])

In [ ]:
Stats[2].shape, Stats[3].shape

In [ ]:
candid_coor=np.int32(Stats[3])
min_row,min_col = origins[tile]
candid_coor

In [ ]:
pos_coor=np.int32(df[['y','x']])
pos_coor -= np.array([min_row,min_col])
pos_coor.shape

In [ ]:
from numpy.linalg import norm
Dists=[]
for i in range(len(candid_coor)):
    c=norm(pos_coor-candid_coor[i],axis=1)
    Dists.append((i,np.min(c),np.argmin(c)))
Dists

In [ ]:
SD=np.sort([d[1] for d in Dists])
plt.plot(SD[:200])
plt.grid()

In [ ]:
candid_coor.shape,pos_coor.shape

In [ ]:
seg=Stats[1]
seg.shape

In [ ]:
for i in range(len(pos_coor)):
    c=pos_coor[i]-np.array([min_col,min_row])
    if c[0]<0 or c[0]>=tile_height or c[1]<0 or c[1] >=tile_width:
        continue
    print(np.array([min_col,min_row]),c)
    print(i,seg[c[0],c[1]])

In [ ]:
tile_width,tile_height